# Experiment with RNN

## Set paths, additional functions and import packages

**Set paths and functions**

In [1]:
root_logdir = '../Logs/'
data_dir = '../Data'
figs_dir = 'Figs/'
models_dir = '../Models/'

**Set functions**

In [2]:
# Tensorboard logs name generator
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

**Import packages**

In [3]:
import os
import time
import tensorflow as tf
import keras
from keras import models, layers
import tensorflow_addons as tfa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.utils import shuffle
from itertools import product
from tools import *
from tensorflow.keras import datasets, layers, models, losses, Model
import pickle
from generators import *

In [4]:
from imblearn.metrics import sensitivity_specificity_support
%matplotlib notebook

In [5]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))

Num GPUs: 1


2022-10-20 11:33:13.579261: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-20 11:33:13.731358: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-20 11:33:13.731613: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [6]:
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

## Load data

* X = 64 BSPs
* Y = labels (0-6) (Labels 6-7 are unified)
* Y_model = identifier of the AF model (1 to 13)

In [8]:

n_classes = 3 #1: Rotor/no rotor ; 2: RA/LA/No rotor (2 classes) ; 3: 7 regions (3 classes) + no rotor (8 classes)

X_1channel,Y,Y_model = load_data(data_type='1channelTensor', n_classes=n_classes, subsampling= True, fs_sub=50)


['LA_LIPV_150119', 'LA_LSPV_150113', 'LA_LSPV_150203', 'LA_PLAW_140612', 'LA_PLAW_140711_arm', 'LA_RIPV_150121', 'LA_RSPV_150113', 'LA_RSPV_CAF_150115', 'RA_RAA_141216', 'RA_RAA_141230', 'RA_RAFW_140807', 'RA_RAFW_SAF_140730', 'Sinusal_150629', 'TwoRotors_181219']


ValueError: too many values to unpack (expected 3)

n_batches_train,n_batches_test,train_1gen,test_1gen, train_sigs=generator_batches_autoencoder(X_1channel,Y,Y_model,data_type='1channelTensor',val_percentage=0.2,test_percentage=0.2,input_size=500,SNR=20,val=False, shuffle_batches_train = False)

In [ ]:
np.unique(Y_model)
len(Y_model)
count=0
for i in Y_model:
    if i==1:
        count+=1
print(count)



In [ ]:
print(X_1channel.shape)

def define_image(time_instant, Tensor):
    image=Tensor[time_instant,: ,: ]
    return image
def define_frames(Tensor):
    ims = []
    for i in range(200):
        ims.append([Tensor[i, :, :]])
    return ims
    
        
    


In [ ]:
plt.figure()
image=define_image(5,X_1channel )
plt.imshow(image, cmap='jet')
plt.colorbar()
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(1,10):
    image=define_image(i, X_1channel)
    plt.subplot(10, 1, i)
    plt.imshow(image) #map='jet')
plt.show()




In [ ]:
int(len(X_1channel)*0.7)+1+int(len(X_1channel)*0.25)

In [ ]:
from numpy import reshape
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

x_train= X_1channel[0:int(len(X_1channel)*0.7), :]
x_test= X_1channel[int(len(X_1channel)*0.7)+1:int(len(X_1channel)*0.7)+1+int(len(X_1channel)*0.25), :, :]
x_val= X_1channel[int(len(X_1channel)*0.7)+1+int(len(X_1channel)*0.25):-1, :, :]
x_train_or, x_test_or, x_val_or=x_train, x_test, x_val

#Standardization
scalers = {}
for i in range(x_train.shape[1]):
    scalers[i] = StandardScaler()
    x_train[:, i, :] = scalers[i].fit_transform(x_train[:, i, :]) 

for i in range(x_test.shape[1]):
    x_test[:, i, :] = scalers[i].transform(x_test[:, i, :]) 
    
for i in range(x_val.shape[1]):
    x_val[:, i, :] = scalers[i].transform(x_val[:, i, :]) 
x_train_st=x_train

#Reshape for interpolation
print(x_train.shape, x_test.shape, x_val.shape) 
x_train = reshape(x_train, (len(x_train), 6, 16,1)) 
x_test = reshape(x_test, (len(x_test), 6, 16,1))
x_val = reshape(x_val, (len(x_val), 6, 16,1))
print(x_train.shape, x_test.shape, x_val.shape) 

#Interpolation
x_train = tf.keras.layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x_train)
x_test = tf.keras.layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x_test)
x_val = tf.keras.layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x_val)
print(x_train.shape, x_test.shape) 

#Plot
plt.figure()
plt.subplot(2,1,1)
image=define_image(1000,x_train_or)
plt.imshow(image, cmap='jet')
plt.colorbar()

plt.subplot(2,1,2)
image=define_image(1000,x_train_st )
plt.imshow(image, cmap='jet')
plt.colorbar()
plt.show()

In [ ]:
autoencoder_mode = '2D CNN'
n_batch=500

if autoencoder_mode == '2D CNN':
    encoder = models.Sequential()
    #encoder.add(layers.Input(shape=( 6, 16,1)))
    encoder.add(layers.Conv2D(64, (2, 2), strides=1, padding='same', activation='relu', input_shape=x_train.shape[1:]))
    encoder.add(layers.MaxPooling2D((2, 2)))
    encoder.add(layers.Dropout(0.2))
    encoder.add(layers.Conv2D(12, (2, 2), strides=1, padding='same', activation='relu'))
    encoder.add(layers.MaxPooling2D((1, 2)))
    encoder.summary()
    decoder = models.Sequential()
    decoder.add(layers.Conv2D(12, (2,2), strides=1, padding='same', activation='relu', input_shape=encoder.output.shape[1:]))
    decoder.add(layers.UpSampling2D((1, 2)))
    decoder.add(layers.Dropout(0.2))
    decoder.add(layers.Conv2D(64, (2,2), strides=1, padding='same', activation='relu'))
    decoder.add(layers.UpSampling2D((2, 2)))
    decoder.add(layers.Conv2D(1, (2,2), strides=1, padding='same', activation='linear'))
    decoder.summary()
    

    
    

In [ ]:
encoder = models.Sequential()
#encoder.add(layers.Input(shape=( 6, 16,1)))
encoder.add(layers.Conv2D(64, (2, 2), strides=1, padding='same', activation='relu', input_shape=x_train.shape[1:]))
encoder.add(layers.MaxPooling2D((2, 2)))
encoder.add(layers.Dropout(0.2))
encoder.add(layers.Conv2D(12, (2, 2), strides=1, padding='same', activation='relu'))
encoder.add(layers.MaxPooling2D((2, 2)))
encoder.add(layers.Conv2D(12, (2, 2), strides=1, padding='same', activation='relu'))
encoder.add(layers.MaxPooling2D((3, 2)))
encoder.summary()
decoder = models.Sequential()


In [ ]:
decoder.add(layers.Conv2D(12, (2,2), strides=1, padding='same', activation='relu', input_shape=encoder.output.shape[1:]))
decoder.add(layers.UpSampling2D((3, 2)))
decoder.add(layers.Conv2D(12, (2,2), strides=1, padding='same', activation='relu', input_shape=encoder.output.shape[1:]))
decoder.add(layers.UpSampling2D((2, 2)))
decoder.add(layers.Dropout(0.2))
decoder.add(layers.Conv2D(64, (2,2), strides=1, padding='same', activation='relu'))
decoder.add(layers.UpSampling2D((2, 2)))
decoder.add(layers.Conv2D(1, (2,2), strides=1, padding='same', activation='linear'))
decoder.summary()

elif autoencoder_mode == 'ConvLSTM':
    encoder = models.Sequential()
    #encoder.add(layers.Input(shape=(n_batch, 6, 16,1)))
    encoder.add(layers.ConvLSTM2D(64, (2, 2), strides=1, padding='same', activation='relu',input_shape=(6, 16,1), return_sequences=True))
    encoder.add(layers.MaxPooling2D((2, 2)))
    encoder.add(layers.Dropout(0.2))
    #encoder.add(Flatten())
    encoder.add(layers.Conv2D(12, (2, 2), strides=1, padding='same', activation='relu', return_sequences=True))
    encoder.add(layers.MaxPooling2D((1, 2)))
    encoder.summary()
    decoder = models.Sequential()
    decoder.add(layers.Conv2D(12, (2,2), strides=1, padding='same', activation='relu', input_shape=encoder.output.shape[1:], return_sequences=True))
    decoder.add(layers.UpSampling2D((1, 2)))
    decoder.add(layers.Dropout(0.2))
    decoder.add(layers.Conv2D(64, (2,2), strides=1, padding='same', activation='relu', return_sequences=True))
    decoder.add(layers.UpSampling2D((2, 2)))
    decoder.add(layers.Conv2D(1, (2,2), strides=1, padding='same', activation='linear', return_sequences=True))
    decoder.summary()

In [ ]:
conv_autoencoder = Model(inputs=encoder.input, outputs=decoder(encoder.outputs))
conv_autoencoder.compile(optimizer='adam', loss=losses.mean_squared_error)
history = conv_autoencoder.fit(x_train, x_train, batch_size=n_batch, epochs=20, validation_data=(x_test, x_test))

In [ ]:
decoded_imgs = conv_autoencoder.predict(x_val)
print(decoded_imgs.shape, x_val.shape)
decoded_imgs = reshape(decoded_imgs, (len(decoded_imgs), decoded_imgs.shape[1],  decoded_imgs.shape[2]))
x_val = reshape(x_val, (len(x_val),x_val.shape[1],  x_val.shape[2])) 
print(decoded_imgs.shape, x_val.shape)

In [ ]:
# summarize history for loss
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
time_instant= 14000

plt.figure(figsize=(5,5), layout='tight')
plt.subplot(3,1,1)
image1=define_image(time_instant,x_val )
min_val, max_val = np.amin(image1), np.amax(image1)
plt.imshow(image1, vmin=min_val, vmax=max_val)#, cmap='jet')
plt.colorbar()
plt.title('Original')
plt.subplot(3,1,2)
image2=define_image(time_instant,decoded_imgs)
plt.imshow(image2, vmin=min_val, vmax=max_val) #, cmap='jet')
plt.colorbar()
plt.title('Reconstructed')
plt.subplot(3,1,3)
plt.imshow(image2-image1, vmin=min_val, vmax=max_val)
plt.title('Error (Reconstructed-Original)')
plt.colorbar()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(x_val[0:1000, 1, 1], label=('Input'))
plt.plot(decoded_imgs[0:1000, 1, 1], label=('Reconstruction'))
plt.fill_between(np.arange(1000), decoded_imgs[0:1000, 1, 1],x_val[0:1000, 1, 1], color='lightgreen')
plt.legend(labels=["Input", "Reconstruction", "Error"])
plt.show()

# Latent Space 

In [ ]:
latent_vector = encoder.predict(x_val)


In [ ]:
latent_vector.shape